# RAG 시스템 구축 실습

## 학습 목표
- 웹 문서를 활용한 RAG(Retrieval-Augmented Generation) 시스템 구축
- 문서 로딩부터 질의응답까지 전체 파이프라인 이해
- LangChain과 Ollama를 활용한 실제 구현 실습



In [1]:
import bs4
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_ollama import OllamaEmbeddings, OllamaLLM

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
from dotenv import load_dotenv

load_dotenv() # .env. 파일 로드

## 2. 문서 로딩 (Document Loading)

**주요 포인트:**
- `web_paths`: 로딩할 웹 페이지 URL
- `bs_kwargs`: BeautifulSoup 파싱 옵션
- `SoupStrainer`: 특정 HTML 요소만 추출 (불필요한 내용 제거)

In [3]:
# 1. 문서 로딩 (Document Loading)
loader = WebBaseLoader(
    web_paths=("https://www.bbc.com/korean/articles/cl4yml4l6j1o",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["bbc-1cvxiy9", "bbc-fa0wmp"]},
        )
    ),
)
docs = loader.load()
print(f"문서의 수: {len(docs)}")

문서의 수: 1


## 3. 문서 분할 (Document Splitting)

- `chunk_size=500`: 각 청크의 최대 크기 (문자 수)
- `chunk_overlap=50`: 청크 간 중복되는 문자 수 (문맥 보존)
- 분할 이유: LLM의 토큰 제한, 검색 정확도 향상

In [4]:
# 2. 문서 분할 (Splitting)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(docs)
print(f"split size: {len(splits)}")

split size: 9


## 4. 임베딩 및 벡터 저장소 구축

- `OllamaEmbeddings`: 텍스트를 벡터로 변환
- `FAISS`: Facebook이 개발한 효율적인 벡터 검색 라이브러리
- `retriever`: 질문과 유사한 문서 조각을 찾는 검색기

In [5]:
# 3. 임베딩 생성 (Embedding)
embeddings = OllamaEmbeddings(model="daynice/kure-v1")

# 4. 벡터 저장소 구축 (Vector Database)
vector_store = FAISS.from_documents(documents=splits, embedding=embeddings)

# 4-1. 쿼리 저장소 검색을 위한 retriever 생성
retriever = vector_store.as_retriever()

## 5. 프롬프트 템플릿 설정

**프롬프트 설계 원칙:**
- 명확한 역할 정의
- 문맥 기반 답변 지시
- 모르는 것은 모른다고 답하도록 유도
- 추측이나 환각 방지

In [6]:
# PROMPT Template 생성
prompt = PromptTemplate.from_template(
"""당신은 질문-답변(Question-Answering)을 수행하는 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
질문과 관련성이 높은 내용만 답변하고 추측된 내용을 생성하지 마세요. 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

## 6. LLM 초기화 및 체인 생성

- `temperature=0`: 일관된 답변을 위해 창의성 최소화
- `chain`: 프롬프트 → LLM → 출력 파싱의 파이프라인
- `|` 연산자: LangChain의 체인 연결 문법

In [7]:
# Ollama 초기화
llm = OllamaLLM(
    model="exaone3.5:7.8b",
    temperature=0
)

# 체인을 생성합니다.
chain = prompt | llm | StrOutputParser()

## 7. 질의응답 실행

**실행 과정:**
1. **문서 검색**: 질문과 관련된 문서 조각들을 벡터 DB에서 검색
2. **문맥 결합**: 검색된 여러 문서를 하나의 문맥으로 결합
3. **프롬프트 생성**: 질문과 문맥을 템플릿에 삽입
4. **스트리밍 답변**: 실시간으로 답변을 생성하고 출력

In [11]:
while True:
    # 7-1. 사용자의 입력을 기다림
    question = input("\n\n당신: ")
    if question == "끝" or question == "exit":
        break
    # 7-2. 쿼리 처리 (Query-Retriever) : 벡터 DB 에서 참고할 문서 검색
    retrieved_docs = retriever.invoke(question)
    print(f"retrieved size: {len(retrieved_docs)}")
    combined_docs = "\n\n".join(doc.page_content for doc in retrieved_docs)

    # 7-3. 검색된 문서를 첨부해서 PROMPT 생성
    formatted_prompt = {"question": question, "context": combined_docs}
    print(len(combined_docs))
#     # 7-4 체인을 실행하고 결과를 stream 형태로 출력
#     result = ""
#     for chunk in chain.stream(formatted_prompt):
#         print(chunk, end="", flush=True)
#         result += chunk

    # 7-4. 체인을 실행
    result = chain.invoke(formatted_prompt)
    print(result)

retrieved size: 4
1998
극한 호우의 주요 원인으로는 다음과 같은 요소들이 언급되었습니다:

1. **저기압 시스템**: 서해에서 유입된 저기압이 장마전선과 결합하여 많은 강수량을 발생시켰습니다. 특히, 중국과 몽골 지역에서 발생한 저기압이 강하게 발달하여 북태평양 고기압과 충돌하면서 폭우를 초래했습니다.

2. **기후 변화**: 민 교수는 현재의 강우 패턴 변화가 단순히 온도 상승으로 인한 수증기 증가만으로 설명하기 어려울 정도로 심각하다고 지적했습니다. 특히, 최근 몇 년간 바다의 온도 상승이 강우 패턴에 영향을 미치고 있다는 분석이 제기되었습니다.

3. **강수 패턴 변화**: 장 교수는 짧은 시간 동안 집중적으로 비가 내리는 경향이 증가하고 있다고 설명했습니다. 이는 과거와 비교해 짧은 기간에 많은 양의 비가 내리는 패턴이 더 흔해졌음을 의미합니다.

현재 학계에서는 이러한 현상의 정확한 원인을 완전히 규명하지 못하고 있으며, 기후 변화와 대기 상태의 복잡한 상호작용을 연구 중입니다.
retrieved size: 4
1906
주어진 문맥에서 서울 신월동에 건설된 대심도 빗물터널의 수에 대한 구체적인 착공 여부나 추가 건설 계획에 대한 최신 업데이트는 제공되지 않았습니다. 다만, 문맥은 서울시가 상습 침수 지역에 대심도 빗물터널 6개를 추가로 건설하겠다고 발표했으나, 고금리와 건설경기 불황 등으로 인해 아직 한 곳도 착공하지 못한 상황임을 언급하고 있습니다. 따라서, 현재까지 신월동에 건설된 빗물터널의 수는 **한 곳**이지만, 추가 건설 계획은 진행 중이지만 아직 완료되지 않았음을 알 수 있습니다.

**답변 요약:**
- 현재 서울 신월동에 건설된 대심도 빗물터널은 **한 곳**입니다.
- 서울시는 추가로 6개를 건설할 계획이지만, 고금리와 건설 경기 불황으로 인해 아직 착공되지 않았습니다.
